### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 使用 colab 環境的同學請執行以下程式碼
%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
import tensorflow as tf
print(tf.__version__)

import os
from google.colab import drive 
drive.mount('/content/drive') # 將 google drive 掛載在 colob，

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%cd '/content/drive/My Drive/cv/day42/cupoy_cv_part4'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

/content/drive/My Drive/cv/day42/cupoy_cv_part4


In [21]:
!ls

IdLookupTable.csv     test.csv	training.csv
SampleSubmission.csv  test.zip	training.zip


In [0]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [23]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [24]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

Using TensorFlow backend.


In [25]:
# 定義人臉關鍵點檢測網路
model = Sequential()

# 定義神經網路的輸入, hidden layer 以及輸出

model.add(Conv2D(filters = 128 , kernel_size = (5 , 5) , strides = (1 , 1) , padding = 'same' , input_shape = (96 , 96 , 1) ))

model.add(Conv2D(filters = 64 , kernel_size = (1 , 1) , strides = (1 , 1) , padding = 'same' ))
model.add(Conv2D(filters = 64 , kernel_size = (3 , 3) , strides = (1 , 1) , padding = 'same' ))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2) ))

model.add(Conv2D(filters = 64 , kernel_size = (1 , 1) , strides = (1 , 1) , padding = 'same' ))
model.add(Conv2D(filters = 64 , kernel_size = (3 , 3) , strides = (1 , 1) , padding = 'same' ))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2) ))


model.add(Conv2D(filters = 32 , kernel_size = (1 , 1) , strides = (1 , 1) , padding = 'same' ))
model.add(Conv2D(filters = 32 , kernel_size = (3 , 3) , strides = (1 , 1) , padding = 'same' ))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2) ))


model.add(Flatten())
model.add(Dense(units = 64 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 30))

# 配置 loss funtion 和 optimizer
model.compile(loss='mse', optimizer='adam')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 128)       3328      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        8256      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 64)        4160      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)       